# Analyze 

In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import datetime
import json
import folium
import utils
from utils import categorize_time_to_half_hour, categorize_time_to_half_hour_numerical, dt_to_time, time_to_time
import consts

## Defines

In [ ]:
USE_GRID = 'hex'

In [ ]:
if USE_GRID == 'nuts':
    path_timedata = consts.PATH_TIMEDATA_NUTS
    path_grid = consts.PATH_MASTERNUTS
    path_map = consts.PATH_DOCS / 'sunrise-nuts.html'
elif USE_GRID == 'hex':
    path_timedata = consts.PATH_TIMEDATA_HEX
    path_grid = consts.PATH_HEXAGON
    path_map = consts.PATH_DOCS / 'sunrise-hex.html'
else:
    raise ValueError(f"Unknown grid type {USE_GRID}")

## Load data

In [21]:
# Load timedata
time_data = json.load(open(path_timedata, 'r', encoding='utf-8'))

# Load Nuts3
gdf_grid_raw = gpd.read_file(path_grid)

# Reduce precision
gdf_grid_raw['geometry'] = gdf_grid_raw['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

# Load Countryborders
gdf_borders = gpd.read_file(consts.PATH_BORDERS)
gdf_borders['geometry'] = gdf_borders['geometry'].apply(lambda geom: geom.simplify(tolerance=0.01))

# Load Timezones
gdf_timezones = gpd.read_file(Path('../data/timezones.geojson'))
gdf_timezones = gdf_timezones.set_crs(epsg=4326)
gdf_timezones = gdf_timezones[['geometry']]

## Prepare Dataset

In [22]:
# Serialize JSON
df_timedata = list(map(lambda x: {
    'nuts_id': x['nuts_id'],
    'name': x['name'],
    'sunrise_current': x['sunrise']['current']['latest'],
    'sunrise_summertime': x['sunrise']['summer']['latest'],
    'sunrise_wintertime': x['sunrise']['winter']['latest'],
    # 'sunrise_current': x['sunrise']['current']['earliest'],
    # 'sunrise_summertime': x['sunrise']['summer']['earliest'],
    # 'sunrise_wintertime': x['sunrise']['winter']['earliest'],
}, time_data))

df_timedata = pd.DataFrame(df_timedata)

# Convert strings to time
df_timedata['sunrise_current'] = df_timedata['sunrise_current'].apply(time_to_time)
df_timedata['sunrise_summertime'] = df_timedata['sunrise_summertime'].apply(time_to_time)
df_timedata['sunrise_wintertime'] = df_timedata['sunrise_wintertime'].apply(time_to_time)

# Current System
df_timedata['sunrise_current_category'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_current_numerical'] = df_timedata['sunrise_current'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Summertime
df_timedata['sunrise_summertime_category'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_summertime_numerical'] = df_timedata['sunrise_summertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Everything on Wintertime
df_timedata['sunrise_wintertime_category'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour
)

df_timedata['sunrise_wintertime_numerical'] = df_timedata['sunrise_wintertime'].apply(
    categorize_time_to_half_hour_numerical
)

# Merge Dataframes
gdf_grid = gdf_grid_raw.merge(df_timedata, left_on='NUTS_ID', right_on='nuts_id')

In [ ]:
m = folium.Map(
    tiles=None,
    prefer_canvas=False
)

folium.TileLayer(
    'cartodbpositron',
    opacity=0.4
).add_to(m)

# Create bins evenly distributed with 0 in the center
min_value = min(
    gdf_grid['sunrise_current_numerical'].min(),
    gdf_grid['sunrise_summertime_numerical'].min(),
    gdf_grid['sunrise_wintertime_numerical'].min()
)
max_value = max(
    gdf_grid['sunrise_current_numerical'].max(),
    gdf_grid['sunrise_summertime_numerical'].max(),
    gdf_grid['sunrise_wintertime_numerical'].max()
)
bins = [x * 0.5 for x in range(int(min_value * 2), int(max_value * 2) + 1)]

def add_choropleth(name, field, popup_fields):

    cp = folium.Choropleth(
        geo_data=gdf_grid.to_json(default=str),
        name=name,
        data=df_timedata,
        columns=["nuts_id", field],
        key_on="feature.properties.NUTS_ID",
        fill_color="Greys",
        fill_opacity=1,
        line_opacity=0.4,
        bins = bins,
        overlay=True,
        show=(name == "Aktuelles System")
    ).add_to(m)


    folium.GeoJsonTooltip(["name"] + popup_fields).add_to(cp.geojson)    

add_choropleth("Aktuelles System", 'sunrise_current_numerical', ['sunrise_current_category', 'sunrise_current'])
add_choropleth("Sommerzeit", 'sunrise_summertime_numerical', ['sunrise_summertime_category', 'sunrise_summertime'])
add_choropleth("Winterzeit", 'sunrise_wintertime_numerical', ['sunrise_wintertime_category', 'sunrise_wintertime'])

# Add Country borders
boudaries = folium.GeoJson(
    gdf_borders,
    name="Ländergrenzen",
    style_function=lambda x: {
        'color': 'white',
        'weight': 1,
        'fillOpacity': 0,
        'interactive':False
    },
    control=False
).add_to(m)
m.keep_in_front(boudaries)

m.fit_bounds([[34.615, -10.334], [71.114, 34.5896]])

folium.LayerControl(collapsed=False).add_to(m)

utils.addTitle(m, "Spätester Sonnenaufgang")

m.save(path_map)

## Create Hourly-Map

In [31]:
# Load timedata
time_data_hourly = json.load(open(consts.PATH_TIMEDATA_HOURLY_HEX, 'r', encoding='utf-8'))

In [32]:
# Create style dict
styledict = {}

colors = {
    'night': 'black',
    'dawn': '#312a7d',
    'sunrise': '#b48ad1',
    'day': '#e1cde1',
}

for cell in time_data_hourly:

    styledict[cell['nuts_id']] = {}
    for step in cell['sunrise']['current']['latest']:

        key = '2025-01-01T{time}'.format(
            time=step['time']
        )
        
        key = int(datetime.datetime.strptime(key, '%Y-%m-%dT%H:%M').timestamp())

        styledict[cell['nuts_id']][key] = {
            'color': colors[step['elevation_category']],
            'opacity': 1,
            'cat': step['elevation_category'],
            'elevation': step['elevation'],
            'time': step['time']
        }

        # e = step['elevation']
        # if e < -6:
        #     color = 'black'
        # elif e < 0: #
        #     color = "#5238a7"
        # elif e < 6:
        #     color = "#764bc5"
        # elif e < 12:
        #     color = "#976acd"
        # elif e < 18:
        #     color = "#b48ad1"
        # else:
        #     color = "#e1cde1"


        # styledict[cell['nuts_id']][key] = {
        #     'color': color,
        #     'opacity': 1
        # }


In [33]:
import folium
from folium.plugins import TimeSliderChoropleth

m = folium.Map()

gdf = gdf_grid.set_index('NUTS_ID')

TimeSliderChoropleth(
    gdf.to_json(default=str),
    styledict=styledict,
    init_timestamp=0,
    stroke_width=0,
    date_options='HH:mm'

).add_to(m)

# Add Country borders
boudaries = folium.GeoJson(
    gdf_timezones,
    name="Zeitzonen",
    style_function=lambda x: {
        'color': 'white',
        'weight': 1,
        'fillOpacity': 0,
        'interactive':False
    },
    control=False
).add_to(m)
m.keep_in_front(boudaries)


m.fit_bounds([[34.615, -10.334], [71.114, 34.5896]])

m.save(consts.PATH_DOCS / 'sunrise-hex-hourly.html')

In [29]:
gdf_timezones

,geometry
0,"POLYGON ((44.86486 37.20735, 44.5045 37.20735,..."
1,"POLYGON ((84.14414 60.81098, 83.78378 60.81098..."
2,"POLYGON ((-37.65766 83.69389, -37.65766 90.000..."
3,"POLYGON ((-58.55856 -62.4324, -58.1982 -62.432..."
4,"POLYGON ((157.65766 76.66686, 157.65766 84.955..."
...,...
115,"POLYGON ((172.43243 -77.56755, 170.27027 -77.5..."
116,"POLYGON ((105.04505 -77.38737, 105.04505 -79.5..."
117,"POLYGON ((77.2973 -68.91889, 77.2973 -69.27926..."
118,"POLYGON ((-9.90991 -70.7207, -9.90991 -71.9819..."


In [27]:
gdf_borders

,CNTR_ID,CNTR_NAME,NAME_ENGL,NAME_FREN,ISO3_CODE,SVRG_UN,CAPT,EU_STAT,EFTA_STAT,CC_STAT,NAME_GERM,geometry
0,AS,American Samoa-Sāmoa Amelika,American Samoa,Samoa américaines,ASM,US Non-Self-Governing Territory,Pago Pago,F,F,F,Amerikanisch-Samoa,"MULTIPOLYGON (((-169.55866 -14.20964, -169.426..."
1,AT,Österreich,Austria,Autriche,AUT,UN Member State,Vienna,T,F,F,Österreich,"POLYGON ((15.75363 48.85218, 16.29276 48.73385..."
2,AD,Andorra,Andorra,Andorre,AND,UN Member State,Andorra la Vella,F,F,F,Andorra,"POLYGON ((1.44257 42.60367, 1.53823 42.6412, 1..."
3,AF,افغانستان-افغانستان,Afghanistan,Afghanistan,AFG,UN Member State,Kabul,F,F,F,Afghanistan,"POLYGON ((71.57093 37.87954, 71.46184 36.99849..."
4,AG,Antigua and Barbuda,Antigua and Barbuda,Antigua-et-Barbuda,ATG,UN Member State,St John's,F,F,F,Antigua und Barbuda,"MULTIPOLYGON (((-61.73379 17.04294, -61.80546 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
254,ZM,Zambia,Zambia,Zambie,ZMB,UN Member State,Lusaka,F,F,F,Sambia,"POLYGON ((30.79625 -8.27651, 31.0234 -8.58042,..."
255,ZW,Zimbabwe,Zimbabwe,Zimbabwe,ZWE,UN Member State,Harare,F,F,F,Simbabwe,"POLYGON ((29.82726 -15.61733, 30.29121 -15.647..."
256,LT,Lietuva,Lithuania,Lituanie,LTU,UN Member State,Vilnius,T,F,F,Litauen,"MULTIPOLYGON (((24.88203 56.42898, 25.17127 56..."
257,UM,United States Minor Outlying Islands,United States Minor Outlying Islands,Îles mineures éloignées des États-Unis,UMI,US Territory,None,F,F,F,Kleineren Amerikanischen Überseeinseln,"MULTIPOLYGON (((166.70127 19.26981, 166.63347 ..."
